In [ ]:
import sys
sys.path.append('../')
%reload_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin
import utils.paramUtil as paramUtil
from datasets import Text2MotionDataset
import torch
import clip
import numpy as np
from models.transformer import set_requires_grad
from datasets import build_dataloader

In [4]:
class Temp:
    def __init__(self):
        print('mock:: opt')
        self.checkpoints_dir = '../checkpoints'
        self.dataset_name = 't2m'
        self.name = './precompute_text'
        self.times = 1
        self.debug = False
        self.is_train = True
        self.feat_bias = 25
opt = Temp()

opt.save_root = pjoin(opt.checkpoints_dir, opt.dataset_name, opt.name)
opt.meta_dir = pjoin(opt.save_root, 'meta')

os.makedirs(opt.meta_dir, exist_ok=True)

if opt.dataset_name == 't2m':
    opt.data_root = '../data/HumanML3D'
    opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
    opt.text_dir = pjoin(opt.data_root, 'texts')
    opt.joints_num = 22
    opt.max_motion_length = 196
    dim_pose = 263
    kinematic_chain = paramUtil.t2m_kinematic_chain
elif opt.dataset_name == 'kit':
    opt.data_root = '../data/KIT-ML'
    opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
    opt.text_dir = pjoin(opt.data_root, 'texts')
    opt.joints_num = 21
    dim_pose = 251
    opt.max_motion_length = 196
    kinematic_chain = paramUtil.kit_kinematic_chain

else:
    raise KeyError('Dataset Does Not Exist')

mean = np.load(pjoin(opt.data_root, 'Mean.npy'))
std = np.load(pjoin(opt.data_root, 'Std.npy'))

train_split_file = pjoin(opt.data_root, 'train.txt')
# train_dataset = Text2MotionDataset(opt, mean, std, train_split_file, opt.times)

mock:: opt


### I stopped here. Trying to move out from Text2MotionDataset 
- HumanML3D adds caption in specific duration (start-stop frames) and create new files name on the fly.

In [5]:
import codecs as cs
from tqdm import tqdm
split_file = train_split_file

id_list = []
with cs.open(split_file, 'r') as f:
    for line in f.readlines():
        id_list.append(line.strip())
if opt.debug:
    id_list = id_list[:300]

new_name_list = []
length_list = []
# for name in tqdm(id_list):

In [ ]:

class TextEncoder(torch.nn.Module):
    def __init__(self):
        self.clip, _ = clip.load('ViT-B/32', "cpu")
        set_requires_grad(self.clip, False)
    
    def forward(self, text, device):
        with torch.no_grad():
            # [info] token of text.shape = [b, 77]. The tokens after the text input is "0". "<|startoftext|>" and "<|endoftext|>" are added. The unknow words "asldfjh" takes more than 1 tokens.
            text = clip.tokenize(text, truncate=True).to(device)
            x = self.clip.token_embedding(text).type(self.clip.dtype)  # [batch_size, n_ctx, d_model]

            x = x + self.clip.positional_embedding.type(self.clip.dtype)
            x = x.permute(1, 0, 2)  # NLD -> LND
            x = self.clip.transformer(x)
            x = self.clip.ln_final(x).type(self.clip.dtype)

In [ ]:
for caption, motion, m_length in train_dataset:
    print(caption)
    break